In [7]:
from sklearn.datasets import make_blobs
from sklearn.datasets import make_regression
from matplotlib import pyplot
from pandas import DataFrame
from sklearn import tree
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error as mse
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from mpl_toolkits import mplot3d
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import PredefinedSplit
from hypopt import GridSearch 
from random import randrange
import matplotlib.pyplot as plt
from sklearn.model_selection import KFold
r = tree.DecisionTreeRegressor(max_depth = 3)
from sklearn.utils import resample

In [8]:
n = 500
a1 = 2
a2 = 0.5

<img src="test3.png">

In [20]:
MSE1 = []
MSE2 = []
MSE3 = []
MSE4 = []
for j in range(999):
    x1 = np.random.normal(loc = 0.5, scale = 1.0, size = (n,1))
    x2 = np.random.normal(loc = 0.0, scale = 1.0, size = (n,1))
    x3_ = np.random.normal(loc = 0.5, scale = 1.0, size = (n,1))
    e = np.random.normal(loc = 0.0, scale = 0.8, size = (n,1))
    d = np.random.choice([0, 1.5], size=(n,1), p=[1./3, 2./3])
    x3 = np.multiply(x3_, d)

    y = np.add(x1*a1, x2*a2)
    y = np.add(y, x3)
    y = np.add(y, e)
    X = np.concatenate((x1,x2), axis=1)
    X = np.concatenate((X,x3), axis=1)
    #data = pd.DataFrame({'X1': x1[:,0], 'X2': x2[:,0], 'y': y[:,0]}, columns=['X1', 'X2', 'y'])


    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.5)
    r = r.fit(X_train, y_train)
    yhat = r.predict(X_test)
    mse1 = mse(y_test, yhat)
    MSE1.append(mse1)
    
    
    
    #CV
    kf = KFold(n_splits=5)
    kf.get_n_splits(X)

    mse2 = []
    for train_index, test_index in kf.split(X):
        X_train2, X_test2 = X[train_index], X[test_index]
        y_train2, y_test2 = y[train_index], y[test_index]
        r.fit(X_train2, y_train2)
        yhat2 = r.predict(X_test2)
        mse2.append(mse(y_test2, yhat2))
    MSE2.append(np.mean(mse2))
    
    
    
    data = pd.DataFrame({'X1': X[:,0], 'X2': X[:,1], 'X3': X[:,2], 'y': y[:,0]}, columns=['X1', 'X2', 'X3','y'])
    mse3 = []
    for q in range(5):
        X3, y3 = resample(X, y)
        data_res = pd.DataFrame({'X1': X3[:,0], 'X2': X3[:,1], 'X3': X3[:,2], 'y': y3[:,0]}, columns=['X1', 'X2', 'X3','y'])
        data['condition'] = data['y'].isin(data_res['y'])
        test_sample = data[data['condition'] == False]
        X_test3 = test_sample[['X1', 'X2', 'X3']]
        y_test3 = test_sample['y']
    
        r = r.fit(X3, y3)
        yhat3 = r.predict(X_test3)
        mse3.append(mse(y_test3, yhat3))
    MSE3.append(np.mean(mse3))
    
    
    mse4 = []
    for q in range(5):
        X4, y4 = resample(X, y, n_samples = 100)
        data_res = pd.DataFrame({'X1': X4[:,0], 'X2': X4[:,1], 'X3': X4[:,2], 'y': y4[:,0]}, columns=['X1', 'X2', 'X3','y'])
        data['condition'] = data['y'].isin(data_res['y'])
        train_sample = data[data['condition'] == False]
        X_train3 = train_sample[['X1', 'X2', 'X3']]
        y_train3 = train_sample['y']
    
        r = r.fit(X_train3, y_train3)
        yhat4 = r.predict(X4)
        mse4.append(mse(y4, yhat4))
    MSE4.append(np.mean(mse4))
    
print(np.mean(MSE1))
print(np.mean(MSE2))
print(np.mean(MSE3))
print(np.mean(MSE4))
print(np.std(MSE1))
print(np.std(MSE2))
print(np.std(MSE3))
print(np.std(MSE4))

2.2214577987486788
2.1203676116193932
2.196065097511324
2.1179323319906675
0.2585320602629587
0.1772741305822798
0.177069179111475
0.21501133090252314


In [21]:
results = pd.DataFrame({'1/2 Test-Train Split': [np.mean(MSE1), np.std(MSE1)], '5 Fold CV': [np.mean(MSE2), np.std(MSE2)], 'Resample training data (rep5)': [np.mean(MSE3), np.std(MSE3)], 'Resample test data (rep5)': [np.mean(MSE4), np.std(MSE4)]}, index = ['MSE Mean (MC rep = 1000)', 'MSE St Dev (MC rep = 1000)'])

In [22]:
results

,1/2 Test-Train Split,5 Fold CV,Resample training data (rep5),Resample test data (rep5)
MSE Mean (MC rep = 1000),2.221458,2.120368,2.196065,2.117932
MSE St Dev (MC rep = 1000),0.258532,0.177274,0.177069,0.215011


In [18]:
mse4 = []
X4, y4 = resample(X, y, n_samples = 100)
data_res = pd.DataFrame({'X1': X4[:,0], 'X2': X4[:,1], 'X3': X4[:,2], 'y': y4[:,0]}, columns=['X1', 'X2', 'X3','y'])
data['condition'] = data['y'].isin(data_res['y'])
train_sample = data[data['condition'] == False]
X_train3 = train_sample[['X1', 'X2', 'X3']]
y_train3 = train_sample['y']

r = r.fit(X_train3, y_train3)
yhat4 = r.predict(X4)
mse4.append(mse(y4, yhat4))

In [19]:
len(data[data['condition'] == False])

410

In [ ]:
#set up simple model with heterogeneous causal effects
#treatment -> random
#3-4 X vars (1 relevant for treatment effect D=1 if x>0, D=2 if x<0 etc.)
#use causal tree -> estimate
#it works?

#monte carlo

#investigate: how sample affects the results

#1 split (along relevant X)
